In [58]:
# Import Libraries
import hub as hub
import pandas as pd
import numpy as np
import re

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

In [2]:
# Read Dataset
df_processed = pd.read_csv(r'processed_data.csv')
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5413 entries, 0 to 5412
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          5413 non-null   int64 
 1   Job_Title           5413 non-null   object
 2   Description         5413 non-null   object
 3   lower_description   5413 non-null   object
 4   word_tokenized      5413 non-null   object
 5   sentence_tokenized  5413 non-null   object
 6   word_count          5413 non-null   int64 
 7   sentence_count      5413 non-null   int64 
 8   clean_words         5413 non-null   object
 9   clean_stemmed       5413 non-null   object
 10  clean_lemmed        5413 non-null   object
dtypes: int64(3), object(8)
memory usage: 465.3+ KB


In [3]:
df_processed.head()

,Unnamed: 0,Job_Title,Description,lower_description,word_tokenized,sentence_tokenized,word_count,sentence_count,clean_words,clean_stemmed,clean_lemmed
0,0,Data Scientist,"POSITION SUMMARY, The Business Analyst role is...","position summary, the business analyst role is...","['position', 'summary', 'the', 'business', 'an...","['POSITION SUMMARY, The Business Analyst role ...",424,25,"['position', 'summary', 'business', 'analyst',...","['posit', 'summari', 'busi', 'analyst', 'role'...","['position', 'summary', 'business', 'analyst',..."
1,1,Data Scientist,"What do we need?, You to have an amazing perso...","what do we need?, you to have an amazing perso...","['what', 'do', 'we', 'need', 'you', 'to', 'hav...","['What do we need?, You to have an amazing per...",286,10,"['need', 'amazing', 'personality', 'communicat...","['need', 'amaz', 'person', 'commun', 'style', ...","['need', 'amazing', 'personality', 'communicat..."
2,2,Data Scientist,"Validate, analyze, and conduct statistical ana...","validate, analyze, and conduct statistical ana...","['validate', 'analyze', 'and', 'conduct', 'sta...","['Validate, analyze, and conduct statistical a...",314,24,"['validate', 'analyze', 'conduct', 'statistica...","['valid', 'analyz', 'conduct', 'statist', 'ana...","['validate', 'analyze', 'conduct', 'statistica..."
3,3,Graduate Studies Program - Data Scientist,"Full time, Washington, DC metro area, Starting...","full time, washington, dc metro area, starting...","['full', 'time', 'washington', 'dc', 'metro', ...","['Full time, Washington, DC metro area, Starti...",297,13,"['full', 'time', 'washington', 'dc', 'metro', ...","['full', 'time', 'washington', 'dc', 'metro', ...","['full', 'time', 'washington', 'dc', 'metro', ..."
4,4,Data Scientist I,Assist in consultations with business partners...,assist in consultations with business partners...,"['assist', 'in', 'consultations', 'with', 'bus...",['Assist in consultations with business partne...,316,7,"['assist', 'consultations', 'business', 'partn...","['assist', 'consult', 'busi', 'partner', 'inte...","['assist', 'consultation', 'business', 'partne..."


In [4]:
# Select job_title and lemmed
col_names = ["Job_Title", "clean_lemmed"]
df_lemmed = df_processed[col_names]
df_lemmed.head()

,Job_Title,clean_lemmed
0,Data Scientist,"['position', 'summary', 'business', 'analyst',..."
1,Data Scientist,"['need', 'amazing', 'personality', 'communicat..."
2,Data Scientist,"['validate', 'analyze', 'conduct', 'statistica..."
3,Graduate Studies Program - Data Scientist,"['full', 'time', 'washington', 'dc', 'metro', ..."
4,Data Scientist I,"['assist', 'consultation', 'business', 'partne..."


In [5]:
df_lemmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5413 entries, 0 to 5412
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job_Title     5413 non-null   object
 1   clean_lemmed  5413 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB


In [6]:
vc = df_lemmed["Job_Title"].value_counts()

In [7]:
df_new = vc[vc > 40]

In [8]:
print(df_new)

Data Scientist              663
Data Engineer               375
Data Analyst                373
Senior Data Scientist       197
Senior Data Engineer        135
Senior Data Analyst          82
Big Data Engineer            78
Principal Data Scientist     60
Sr. Data Scientist           43
Lead Data Scientist          42
Name: Job_Title, dtype: int64


In [9]:
df_10 = df_lemmed.loc[df_lemmed['Job_Title'].isin(["Data Scientist", "Data Engineer", "Data Analyst", "Senior Data Scientist", "Senior Data Engineer", "Senior Data Analyst", "Big Data Engineer", "Principal Data Scientist", "Sr. Data Scientist", "Lead Data Scientist"])]

In [61]:
df_5 = df_lemmed.loc[df_lemmed['Job_Title'].isin(["Data Scientist", "Data Engineer", "Data Analyst", "Senior Data Scientist"])]

In [10]:
df_10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2048 entries, 0 to 5408
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job_Title     2048 non-null   object
 1   clean_lemmed  2048 non-null   object
dtypes: object(2)
memory usage: 48.0+ KB


In [11]:
df1 = df_10[df_10["Job_Title"]=="Data Scientist"]
df2 = df_10[df_10["Job_Title"]=="Data Engineer"]
df3 = df_10[df_10["Job_Title"]=="Data Analyst"]
df4 = df_10[df_10["Job_Title"]=="Senior Data Scientist"]
df5 = df_10[df_10["Job_Title"]=="Senior Data Engineer"]
df6 = df_10[df_10["Job_Title"]=="Senior Data Analyst"]
df7 = df_10[df_10["Job_Title"]=="Big Data Engineer"]
df8 = df_10[df_10["Job_Title"]=="Principal Data Scientist"]
df9 = df_10[df_10["Job_Title"]=="Sr. Data Scientist"]
df10 = df_10[df_10["Job_Title"]=="Lead Data Scientist"]


In [12]:
# df1_down_sampled = df1.sample(n= 42)
# df2_down_sampled = df2.sample(n= 42)
# df3_down_sampled = df3.sample(n= 42)
# df4_down_sampled = df4.sample(n= 42)
# df5_down_sampled = df5.sample(n= 42)
# df6_down_sampled = df6.sample(n= 42)
# df7_down_sampled = df7.sample(n= 42)
# df8_down_sampled = df8.sample(n= 42)
# df9_down_sampled = df9.sample(n= 42)
# df10_down_sampled = df10.sample(n= 42)

In [62]:
df1_down_sampled = df1.sample(n= 100)
df2_down_sampled = df2.sample(n= 100)
df3_down_sampled = df3.sample(n= 100)
df4_down_sampled = df4.sample(n= 100)
df5_down_sampled = df5.sample(n= 100)

In [13]:
df1_down_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 501 to 1287
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job_Title     42 non-null     object
 1   clean_lemmed  42 non-null     object
dtypes: object(2)
memory usage: 1008.0+ bytes


In [63]:
df_down_sampled = pd.concat([df1_down_sampled, df2_down_sampled, df3_down_sampled, df4_down_sampled, df5_down_sampled])

In [64]:
df_down_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1028 to 4913
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job_Title     500 non-null    object
 1   clean_lemmed  500 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


In [65]:
df_down_sampled['Job_Title'].value_counts()

Data Scientist           100
Data Engineer            100
Data Analyst             100
Senior Data Scientist    100
Senior Data Engineer     100
Name: Job_Title, dtype: int64

In [66]:
y = preprocessing.LabelEncoder()
y.fit(df_down_sampled.Job_Title)
label_encoded = y.transform(df_down_sampled.Job_Title)
label_encoded_df = pd.DataFrame(label_encoded, columns=['JobTitle_Encoded'])

In [67]:
label_encoded_df.head()

,JobTitle_Encoded
0,2
1,2
2,2
3,2
4,2


In [68]:
df_encoded = df_down_sampled.reset_index(drop=True)

In [69]:
df_encoded = df_encoded.join(label_encoded_df)

In [71]:
df_encoded

,Job_Title,clean_lemmed,JobTitle_Encoded
0,Data Scientist,"['science', 'technology', 'mission', '60', 'ye...",2
1,Data Scientist,"['utilizing', 'feature', 'billion', 'transacti...",2
2,Data Scientist,"['design', 'implement', 'data', 'analysis', 'd...",2
3,Data Scientist,"['master', 'phd', 'degree', 'industrial', 'eng...",2
4,Data Scientist,"['symantec', 'cloudsoc', 'leader', 'data', 'sc...",2
...,...,...,...
495,Senior Data Engineer,"['capital', 'one', 'building', 'leading', 'inf...",3
496,Senior Data Engineer,"['requisition', 'industry', 'wb', 'business', ...",3
497,Senior Data Engineer,"['description', 'responsibility', 'senior', 's...",3
498,Senior Data Engineer,"['5', 'year', 'experience', 'data', 'engineeri...",3


In [72]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2),
                        )

In [73]:
# We transform each complaint into a vector
features = tfidf.fit_transform(df_encoded.clean_lemmed).toarray()

In [74]:
labels = df_encoded.JobTitle_Encoded

In [75]:
features.shape
JobTitle_Encoded_df = df_encoded[['Job_Title', 'JobTitle_Encoded']].drop_duplicates()
# Dictionaries
Encoded_to_Title = dict(JobTitle_Encoded_df.values)
Title_to_Encoded = dict(JobTitle_Encoded_df[['JobTitle_Encoded', 'Job_Title']].values)

In [76]:
# Finding the three most correlated terms with each of the product categories
N = 3
for Job_Title, JobTitle_Encoded in sorted(Encoded_to_Title.items()):
  features_chi2 = chi2(features, labels == JobTitle_Encoded)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("n==> %s:" %(Job_Title))
  print("  * Most Correlated Unigrams are: %s" %(', '.join(unigrams[-N:])))
  print("  * Most Correlated Bigrams are: %s" %(', '.join(bigrams[-N:])))

C:\Users\Arun\PycharmProjects\JDEngine\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Arun\PycharmProjects\JDEngine\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


n==> Data Analyst:
  * Most Correlated Unigrams are: excel, capgemini, analyst
  * Most Correlated Bigrams are: reporting tool, analyst work, data analyst
n==> Data Engineer:
  * Most Correlated Unigrams are: enhanced, etl, nifi
  * Most Correlated Bigrams are: database including, good data, data engineer
n==> Data Scientist:
  * Most Correlated Unigrams are: supervised, unsupervised, statistical
  * Most Correlated Bigrams are: statistical analysis, experience statistical, conducting research
n==> Senior Data Engineer:
  * Most Correlated Unigrams are: engineer, lambda, streaming
  * Most Correlated Bigrams are: data pipeline, data warehouse, architect build
n==> Senior Data Scientist:
  * Most Correlated Unigrams are: machine, nlp, ph
  * Most Correlated Bigrams are: learning framework, machine learning, leadership team


C:\Users\Arun\PycharmProjects\JDEngine\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Arun\PycharmProjects\JDEngine\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Arun\PycharmProjects\JDEngine\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [77]:
X = df_encoded['clean_lemmed']
y = df_encoded['Job_Title']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state = 0)

In [78]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

In [79]:
# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

In [80]:
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [81]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1,
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LinearSVC,0.622,0.044944
LogisticRegression,0.626,0.027019
MultinomialNB,0.608,0.030332
RandomForestClassifier,0.596,0.039115
